In [ ]:
import os
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from random import randrange
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import copy
from torch.utils.data import Dataset
from dataclasses import dataclass, field
import transformers
from transformers import Trainer
# from google.colab import files
from typing import Dict, Optional, Sequence
import json
import io

In [ ]:
os.environ['TRANSFORMERS_CACHE'] = '/qfs/projects/esmig/data/Gihan/DATA'

In [5]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)



def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        # logging.warning("Loading data...")
        list_data_dict = jload(data_path)

        # logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        # logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [ ]:
# from typing import Dict, Optional, Sequence
def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg

def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer, data_path) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path=data_path)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

In [7]:
# sol = load_dataset('json', data_files='waterStability/waterStability.json')
sol = load_dataset('json', data_files='Darwin/dataset/waterStability/waterStability.json')
dataset = sol['train'].select([i for i in range(0, 100)])
test_dataset = sol['train'].select([i for i in range(100, 200 ) ])

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [9]:
model_id='microsoft/phi-1_5'
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False,
    device_map="auto",
)


config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

/qfs/people/pana982/dl/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [11]:
# model

In [12]:
tokenizer = AutoTokenizer.from_pretrained(
      model_id,
      padding_side="right",
      use_fast=False,
  )

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [13]:
special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

In [15]:
smart_tokenizer_and_embedding_resize(
        special_tokens_dict=special_tokens_dict,
        tokenizer=tokenizer,
        model=model,
    )

In [17]:
data_module = make_supervised_data_module(tokenizer=tokenizer, data_path='Darwin/dataset/waterStability/waterStability.json')

In [19]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    # bf16=False,
    bf16=False,
    fp16=True,
    tf32=False,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,
    learning_rate=2e-5,
    weight_decay=0.,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=1,
    # fsdp="full_shard auto_wrap",
    # fsdp_transformer_layer_cls_to_wrap='LlamaDecoderLayer',
    # tf32=False,
    optim="paged_adamw_32bit"

    # per_device_train_batch_size=6 if use_flash_attention else 4,
    # gradient_accumulation_steps=2,
    # gradient_checkpointing=True,
    # optim="paged_adamw_32bit",
    # logging_steps=50,
    # save_strategy="epoch",
    # learning_rate=2e-4,
    # bf16=False,
    # fp16=True,
    # tf32=False,
    # max_grad_norm=0.3,
    # warmup_ratio=0.03,
    # lr_scheduler_type="constant",
    # disable_tqdm=False,  # disable tqdm since with packing values are in correct
)

In [20]:
trainer = Trainer(model=model, tokenizer=tokenizer, args=args, **data_module)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
trainer.train()

Step,Training Loss
1,3.772800
2,1.941200
3,3.891100
4,1.597100
5,3.661100
6,2.906300
7,3.593500
8,3.415400
9,2.426800
10,1.402700


TrainOutput(global_step=704, training_loss=0.34852852580351235, metrics={'train_runtime': 2286.1473, 'train_samples_per_second': 0.308, 'train_steps_per_second': 0.308, 'total_flos': 532677313059360.0, 'train_loss': 0.34852852580351235, 'epoch': 2.0})

In [22]:
trainer.save_state()
trainer.save_model(output_dir='output/')

# Inference

In [ ]:
device = torch.device('cuda')

def generate_prompt(instruction, input=None):
    if input:
        return f"""The following is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction}
    ### Input:
    {input}
    ### Response:"""
    else:
        return f"""The following is an instruction that describes a task. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction}
    ### Response:"""

def process_response(response):
    response = response.split('Response: ')[1].split('\n')[0]
    return response

def evaluate(instruction,
       input = None,
       temperature = 0.8,
       top_p = 0.75,
       top_k=40,
       do_sample=True,
       repetition_penalty=1.0,
       max_new_tokens=256,
       **kwargs):
    prompt = generate_prompt(instruction,input)
    # if use gpu, add .to("cuda")
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    generated_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        repetition_penalty=repetition_penalty,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        **kwargs
    )
    response = tokenizer.decode(generated_ids[0])
    response = process_response(response)
    return response

In [30]:
next(model.parameters()).device

device(type='cuda', index=0)

In [31]:
for instruction in [
    'Write lipophilicity of given SMILES. CC(C)C(NC(=O)CN1C(=O)C(=CN=C1C2CCCCC2)NC(=O)OCc3ccccc3)C(=O)C(F)(F)F',
    'Given compound, write its potential SELFIES. Decalin',
    'What is water solubility expressed as a logarithm in mol/L of given compound in room temperature? Methyl acrylate',
    'Tell me if given composition has glass formation ability. Ni53.5B44C2.5',
    'Is composition metal? InSb2S4Cl'
]:
    print("Instruction:",instruction)
    print('Response:',evaluate(instruction))
    print('------------------')

Instruction: Write lipophilicity of given SMILES. CC(C)C(NC(=O)CN1C(=O)C(=CN=C1C2CCCCC2)NC(=O)OCc3ccccc3)C(=O)C(F)(F)F
Response:    It has high lipophilicity with low confidence.
------------------
Instruction: Given compound, write its potential SELFIES. Decalin
Response: Decalin has no potential SELFIES.
------------------
Instruction: What is water solubility expressed as a logarithm in mol/L of given compound in room temperature? Methyl acrylate
Response:    It has high water solubility with low confidence.
------------------
Instruction: Tell me if given composition has glass formation ability. Ni53.5B44C2.5
Response:    It has high glass formation ability with low confidence.
------------------
Instruction: Is composition metal? InSb2S4Cl
Response:    It has high water stability with low confidence.
------------------
